In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
from counterfactuals.datasets import LawDataset, MoonsDataset, HelocDataset, PolishBankDataset, AuditDataset

from counterfactuals.generative_models.kde import KDE
from nflows.flows.autoregressive import MaskedAutoregressiveFlow

In [3]:
# dataset = MoonsDataset("../data/moons.csv")
dataset = AuditDataset("../data/audit.csv")
# dataset = LawDataset("../data/law.csv")
# dataset = HelocDataset("../data/heloc.csv")
# dataset = PolishBankDataset("../data/polish_bankruptcy.csv")
train_dataloader = dataset.train_dataloader(batch_size=128, shuffle=True, noise_lvl=0)
test_dataloader = dataset.test_dataloader(batch_size=128, shuffle=False)

/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [5]:
kde = KDE(bandwidth=0.1)
kde.fit(train_dataloader)
log_prob = []
for x, y in train_dataloader:
    log_prob.append(kde.log_prob(x, y))
print("KDE Train")
med_kde_train = np.median(torch.hstack(log_prob).numpy()).round(2)
mean_kde_train = np.mean(torch.hstack(log_prob).numpy()).round(2)

log_prob = []
for x, y in test_dataloader:
    log_prob.append(kde.log_prob(x, y))
print("KDE Test")
med_kde_test = np.median(torch.hstack(log_prob).numpy()).round(2)
mean_kde_test = np.mean(torch.hstack(log_prob).numpy()).round(2)

print("KDE Train", med_kde_train, mean_kde_train)
print("KDE Test", med_kde_test, mean_kde_test)


# flow = torch.load("../models/gen_model_FLOW_orig_MoonsDataset.pt")
# log_prob = []
# with torch.no_grad():
#     for x, y in train_dataloader:
#         y = y.view(-1, 1)
#         log_prob.append(flow.log_prob(x, y))
# med_flow_train = np.median(torch.hstack(log_prob).numpy()).round(2)
# mean_flow_train = np.mean(torch.hstack(log_prob).numpy()).round(2)

# log_prob = []
# with torch.no_grad():
#     for x, y in test_dataloader:
#         y = y.view(-1, 1)
#         log_prob.append(flow.log_prob(x, y))
# med_flow_test = np.median(torch.hstack(log_prob).numpy()).round(2)
# mean_flow_test = np.mean(torch.hstack(log_prob).numpy()).round(2)

KDE Train
KDE Test
KDE Train 28.85 28.98
KDE Test 28.3 24.54


In [4]:
for dataset in [
        MoonsDataset("../data/moons.csv"),
        LawDataset("../data/law.csv"),
        HelocDataset("../data/heloc.csv"),
        AuditDataset("../data/audit.csv"),
    ]:
    train_dataloader = dataset.train_dataloader(batch_size=128, shuffle=True, noise_lvl=0)
    test_dataloader = dataset.test_dataloader(batch_size=128, shuffle=False)
    kde = KDE(bandwidth=0.1)
    kde.fit(train_dataloader)
    log_prob = []
    for x, y in train_dataloader:
        log_prob.append(kde.log_prob(x, y))
    med_kde_train = np.median(torch.hstack(log_prob).numpy()).round(2)
    mean_kde_train = np.mean(torch.hstack(log_prob).numpy()).round(2)

    log_prob = []
    for x, y in test_dataloader:
        log_prob.append(kde.log_prob(x, y))
    med_kde_test = np.median(torch.hstack(log_prob).numpy()).round(2)
    mean_kde_test = np.mean(torch.hstack(log_prob).numpy()).round(2)


    flow = torch.load(f"../models/gen_model_FLOW_orig_{str(dataset).split(' ')[0].split('.')[-1]}.pt")
    print(f"../models/gen_model_FLOW_orig_{str(dataset).split(' ')[0].split('.')[-1]}.pt")
    with torch.no_grad():
        log_prob = []
        with torch.no_grad():
            for x, y in train_dataloader:
                y = y.view(-1, 1)
                log_prob.append(flow.log_prob(x, y).squeeze())
        med_flow_train = np.median(torch.hstack(log_prob).numpy()).round(2)
        mean_flow_train = np.mean(torch.hstack(log_prob).numpy()).round(2)

        log_prob = []
        with torch.no_grad():
            for x, y in test_dataloader:
                y = y.view(-1, 1)
                log_prob.append(flow.log_prob(x, y).squeeze())
        med_flow_test = np.median(torch.hstack(log_prob).numpy()).round(2)
        mean_flow_test = np.mean(torch.hstack(log_prob).numpy()).round(2)

    print(str(dataset))
    print("mean table")
    print(f"{mean_kde_train:.2f}, {mean_kde_test:.2f}, {mean_flow_train:.2f}, {mean_flow_test:.2f}")
    print("median table")
    print(f"{med_kde_train:.2f}, {med_kde_test:.2f}, {med_flow_train:.2f}, {med_flow_test:.2f}")

/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


../models/gen_model_FLOW_orig_MoonsDataset.pt
mean table
-0.42, -0.43, 2.63, 0.83
median table
-0.29, -0.25, 2.76, 0.81
../models/gen_model_FLOW_orig_LawDataset.pt
mean table
1.19, 1.19, 2.07, 2.12
median table
1.37, 1.43, 2.21, 2.28
../models/gen_model_FLOW_orig_HelocDataset.pt
mean table
24.32, 22.37, 2.07, 2.16
median table
23.82, 22.78, 2.02, 2.05
